In [ ]:
from functools import wraps
from inspect import signature
from pydantic import BaseModel, fields

def custom_decorator(func):
    sig = signature(func)
    
    @wraps(func)
    def wrapper( *args, **kwargs):
        # 用函式參數名稱決定要傳入哪些 query 參數
        print("Query parameters kwargs:")
        for name, value in kwargs.items():
            print(f"  {name}: {value}")
            if isinstance(value, BaseModel):
                print(f"  {name} is a Pydantic model")
        print("Query parameters args:")
        for name, value in zip(sig.parameters.keys(), args):
            print(f"  {name}: {value}")
            if isinstance(value, BaseModel):
                print(f"  {name} is a Pydantic model")

        for name, param in sig.parameters.items():
            print(f"Processing parameter: {name} with type {param.annotation}")

        # 呼叫原始的路由函式，傳入抓到的參數
        return func(*args, **kwargs)
    return wrapper

In [ ]:
class MyData:
    def __init__(self, param1: str, param2: int):
        self.param1 = param1
        self.param2 = param2

In [ ]:
class MyData2(BaseModel):
    param1: str
    param2: int

In [ ]:
@custom_decorator
def my_route_function(data: MyData):
    # 這裡是路由函式的實作
    return {"param1": data.param1, "param2": data.param2}

In [ ]:
my_route_function(MyData("test", 123))

In [ ]:
@custom_decorator
def my_route_function2(data: MyData2):
    # 這裡是路由函式的實作
    return {"param1": data.param1, "param2": data.param2}

In [ ]:
my_route_function2(MyData2(param1="test", param2=123))

# Task with Pydantic Test

In [1]:
from skalds.model.task import Task
from skalds.model.event  import UpdateTaskWorkerEvent
from skalds.worker.baseclass import BaseTaskWorker
from datetime import datetime
from typing import Optional
from pydantic import BaseModel, Field

Logger 'llmbrick' (ID: llmbrick): Console only mode


In [2]:
class MyModel(BaseModel):
    id: str
    name: str
    description: Optional[str] = None
    status: int = 0
    created_at: int = Field(default_factory=lambda: int(datetime.utcnow().timestamp()*1000))
    updated_at: int = Field(default_factory=lambda: int(datetime.utcnow().timestamp()*1000))


In [6]:
my_model = MyModel(
    id="1",
    name="Sample Model",
    description="This is a sample model",
    status=0,
    created_at=int(datetime.utcnow().timestamp()*1000),
    updated_at=int(datetime.utcnow().timestamp()*1000)
)

MyModel.model_validate(my_model)


my_task = Task(
    id="1",
    class_name="Sample Task",
    source="User",
    name="Sample Task",
    description="This is a sample task",
    executor="user@example.com",
    create_date_time=int(datetime.utcnow().timestamp()*1000),
    update_date_time=int(datetime.utcnow().timestamp()*1000),
    priority=5,
    attachments=my_model
)

In [4]:
my_task.model_dump()

{'id': '1',
 'class_name': 'Sample Task',
 'source': 'User',
 'name': 'Sample Task',
 'description': 'This is a sample task',
 'executor': 'user@example.com',
 'dependencies': None,
 'mode': 'Passive',
 'create_date_time': 1754973691328,
 'update_date_time': 1754973691328,
 'deadline_date_time': 1755607291328,
 'lifecycle_status': 'Created',
 'priority': 5,
 'attachments': {'id': '1',
  'name': 'Sample Model',
  'description': 'This is a sample model',
  'status': 0,
  'created_at': 1754973691328,
  'updated_at': 1754973691328}}

In [5]:
data = my_task.model_dump()
attachments = data.get('attachments', None)
attachments = MyModel.model_validate(attachments)
data['attachments'] = attachments
my_task2 = Task.model_validate(data)

In [ ]:
from skalds.worker.baseclass import update_event_handler
class MyWorker(BaseTaskWorker[MyModel]):
    def initialize(self, data: MyModel) -> None:
        self.data = data

    @classmethod
    def get_generic(cls) -> BaseModel:
        return cls.__orig_bases__[0].__args__[0]
    
    @update_event_handler
    def handle_update(self, data: MyModel) -> None:
        self.data = data

In [ ]:
MyWorker.get_generic()

In [ ]:
from inspect import signature
sig = signature(MyWorker.initialize)
# isinstance(MyWorker.initialize.__annotations__["data"], BaseModel)
param_annotations = sig.parameters
data_param = param_annotations.get('data').annotation

data_param


In [ ]:
data = my_task.model_dump()
attachments = data.get('attachments', None)

data = MyWorker.get_generic().model_validate(attachments)
data